# EXO 0748-676

In this notebook we will analyze subthreshold data that was saved by BAT around the time of a known flare from the Low Mass X-ray Binary EXO 0748-676. The astronomers telegram post related to the different detections of this source which triggered BAT and those that have subthreshold TTE data associated with them can be found [here](https://www.astronomerstelegram.org/?read=16790). 

Lets start off by importing the different python packages that we will need. 

In [1]:
import xspec
import batanalysis as ba
from swifttools.swift_too import GUANO, ObsQuery, Clock, Data
import swiftbat

import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
from pathlib import Path
from astropy.io import fits
from astropy.time import Time, TimeDelta
import astropy.units as u
from astropy.coordinates import SkyCoord
import datetime
import os

As is usual for our example notebooks, we will save the data in the /tmp/ directory. 

In [2]:
tmp_download_dir="/tmp/batdata/download_examples"

ba.datadir(tmp_download_dir, mkdir=True)

PosixPath('/private/tmp/batdata/download_examples')

Now, we can query the data. We can do this either using one of the trigger numbers provided in the astronomers telegram. Querying with the swiftools package finds this data. If we wanted to download this, then we can simply set `fetch=True`. Downloading data this way misses the necessary housekeeping data necessary to do in depth scientific analyses. The `download_swift_trigger_data` function takes this into account by also searching for the nearest observation with calibration data. 

A more relevant search could be one where a user knows that a flare of some source has occured and they wish to search for subthreshold data around this time. We can use a flare time from the telegram and pass that into `download_swift_trigger_data` to search for the subthreshold TTE data around that time. We also pass in `timewindow=400` to denote that we want to also download the nearest observation with calibration/attitude data that is within 400 seconds of the time of interest. 

In [3]:
trignum=1237121
Data(obsid=f"{trignum:08d}000", fetch=False, subthresh=True)

01237121000,sw01237121000msbal.fits.gz,BAT subthreshold trigger BAT file
'',sw01237121000msbat.hk.gz,BAT subthreshold trigger BAT housekeeping
'',sw01237121000msbevshsp_uf.evt.gz,BAT subthreshold trigger BAT slew/point unfiltered event file
'',sw01237121000msbevtlsp.hk.gz,BAT subthreshold trigger BAT housekeeping
'',sw01237121000msbevtssp.hk.gz,BAT subthreshold trigger BAT housekeeping
'',sw01237121000msbhd.hk.gz,BAT subthreshold trigger BAT housekeeping
'',sw01237121000msbno.fits.gz,BAT subthreshold trigger BAT file
'',sw01237121000mspob.cat.gz,BAT subthreshold trigger catalog


In [4]:
t=Time("2024-07-29T12:05:16")
result = ba.download_swift_trigger_data(triggertime=t.datetime, timewindow=400)
print(result)

/Users/tparsota/miniforge3/lib/python3.12/site-packages/swifttools/swift_too/swift_data.py:394: UserWarning: SWIFT_TLE_ARCHIVE.txt.24219.58610885.gz exists and not overwritten (set clobber=True to override this).
/Users/tparsota/miniforge3/lib/python3.12/site-packages/swifttools/swift_too/swift_data.py:394: UserWarning: sw03104687002pat.fits.gz exists and not overwritten (set clobber=True to override this).
/Users/tparsota/miniforge3/lib/python3.12/site-packages/swifttools/swift_too/swift_data.py:394: UserWarning: sw03104687002pjb.par.gz exists and not overwritten (set clobber=True to override this).
/Users/tparsota/miniforge3/lib/python3.12/site-packages/swifttools/swift_too/swift_data.py:394: UserWarning: sw03104687002pob.cat.gz exists and not overwritten (set clobber=True to override this).
/Users/tparsota/miniforge3/lib/python3.12/site-packages/swifttools/swift_too/swift_data.py:394: UserWarning: sw03104687002ppr.par.gz exists and not overwritten (set clobber=True to override this)

{np.int32(1245445): [Swift_Data(username='anonymous',obsid='01245445000',quicklook='False',auxil='True',subthresh='True',itsdc='False'), Swift_Data(username='anonymous',obsid='03104687002',quicklook='False',auxil='True',bat='True')], np.int32(1245446): [Swift_Data(username='anonymous',obsid='01245446000',quicklook='False',auxil='True',subthresh='True',itsdc='False'), Swift_Data(username='anonymous',obsid='03104687002',quicklook='False',auxil='True',bat='True')]}


/Users/tparsota/miniforge3/lib/python3.12/site-packages/swifttools/swift_too/swift_data.py:394: UserWarning: SWIFT_TLE_ARCHIVE.txt.24219.58610885.gz exists and not overwritten (set clobber=True to override this).
/Users/tparsota/miniforge3/lib/python3.12/site-packages/swifttools/swift_too/swift_data.py:394: UserWarning: sw03104687002pat.fits.gz exists and not overwritten (set clobber=True to override this).
/Users/tparsota/miniforge3/lib/python3.12/site-packages/swifttools/swift_too/swift_data.py:394: UserWarning: sw03104687002pjb.par.gz exists and not overwritten (set clobber=True to override this).
/Users/tparsota/miniforge3/lib/python3.12/site-packages/swifttools/swift_too/swift_data.py:394: UserWarning: sw03104687002pob.cat.gz exists and not overwritten (set clobber=True to override this).
/Users/tparsota/miniforge3/lib/python3.12/site-packages/swifttools/swift_too/swift_data.py:394: UserWarning: sw03104687002ppr.par.gz exists and not overwritten (set clobber=True to override this)

We can see that there are two different subthreshold triggers that are found. These are downloaded into the `tdrss` subdirectory of the `/tmp/batdata/download_examples/` directory that we set earlier. Furthermore, looking in the directory for the trigger number 1245446 (which we know is the one associated with the LMXB source a priori), we see that the `download_swift_trigger_data` function has taken care of setting up the directory for us to load this subthreshold trigger into the `BatEvent` object, allowing us to proceed with our data analysis. 

In [5]:
!ls /tmp/batdata/download_examples/tdrss/2024_07/01245446000/

03104687002                      sw01245446000msbevtlsp.hk.gz
auxil                            sw01245446000msbevtssp.hk.gz
bat                              sw01245446000msbhd.hk.gz
sw01245446000msbal.fits.gz       sw01245446000msbno.fits.gz
sw01245446000msbat.hk.gz         sw01245446000mspob.cat.gz
sw01245446000msbevshsp_uf.evt.gz tdrss


Now, lets load this into the `BatEvent` object. Note that since this is subthreshold TTE data that is saved in a subdirectory of our `datadir()` path, we need to specify that with the `obs_dir` parameter that we pass. 

In [6]:
event=ba.BatEvent(result[1245446][0].obsid, obs_dir=result[1245446][0].entries[0].outdir)

AttributeError: module 'heasoftpy.swift' has no attribute 'utils'

Now, we know that the TTE data is energy calibrated and the associated attitude data is loaded in.

Let us then pretend that we dont know that our LMXB is actually detected in this TTE dataset and follow the steps to try to identify if the source is actually detected. (Even if it is not, we can still use this dataset to place upper limits in the BAT energy range). 

Lets create a skyview from the subthreshold TTE data where the skyview is constructed for 4 energy ranges (15-25, 25-50, 50-100, and 100-350 keV).

In [ ]:
eresolved_skyview=event.create_skyview(energybins=[15,25,50,100,350]*u.keV)

Lets plot the SNR image in the 15-25 keV energy range in SKYX/Y coordinates and in a healpix projection with celestial coordinates. There seem to be a a pixel near the location of the LMXB that has a relatively high SNR. Next we will run the source detection on the skyview. 

In [ ]:
ax = eresolved_skyview.snr_img.plot(emin=15*u.keV, emax=25*u.keV) #, projection="healpix", coordsys="icrs")
plt.savefig(f"{event.result_dir.joinpath("norm_ra_dec_localization.pdf")}")

eresolved_skyview.snr_img.plot(emin=15*u.keV, emax=25*u.keV, projection="healpix", coordsys="icrs")

Detecting the sources in the image using our `detect_sources` method will show us that the LMXB EXO 0748-676 is detected very strongly with CENT_SNR=7.44 in the lowest energy bin. We have just confirmed that the source is detected and are confident in applying the mask weighting for the coordinates of the LMXB. If the source was not detected, we can still follow the mask weighting and spectral fitting steps to place upper limits on the emission from the source in the BAT energy range. 

In [ ]:
eresolved_sources = eresolved_skyview.detect_sources()
c=eresolved_sources["SKYCOORD"][0]
eresolved_sources

Applying the mask weighting is as simple as taking the coordinates for the LMXB and passing them to the `apply_mask_weighting` method. 

In [ ]:
event.apply_mask_weighting(ra=c.ra, dec=c.dec)

With the mask weighting complete, lets construct a lightcurve for this outburst. We will use 1 second time bins and will use our default energy bins. We only plot the single 15-350 keV energy range below. 

We can easily see that this subthreshold TTE dataset is only 10 seconds long which is expected. 

In [ ]:
lc=event.create_lightcurve()
lc.set_timebins(timebinalg="uniform", timedelta=np.timedelta64(1, 's'))

fig, ax = lc.plot(energybins=[15,350]*u.keV)

Lets now plot this relative to the trigger time in teh different energy bands to see the energy resolved behavior of this outburst. We first save the triggertime to the `.trigtime` attribute of our `BatEvent` object. We can now easily plot the energy resolved lightcurve relative to the trigger time. We also plot the exposure fraction to verify that Swift was not slewing during this time period. 

We can see that the LMXB is primarily detected in the 15-25 keV energy band where there is a gradual rise in the counts recieved from the source. 

In [ ]:
event.trigtime=Clock(utctime=t)

lc.plot(plot_relative=True, T0=event.trigtime.mettime, plot_exposure_fraction=True)


We can easily construct time resolved spectra for the same timebins that our lightcurve is binned with. 

In [ ]:
spectra=event.create_pha(tstart=lc.tbins["TIME_START"], tstop=lc.tbins["TIME_STOP"])

We will now fit the spectra serially. We use gaussian statistics (by setting `use_cstat=False`) and fit the spectra with a `cflux*po` model. Looking at the plotted spectra, and the fits, we can see that there aren't any spectra that look very good and the fits are very poor as well. 

In [ ]:
for i in spectra:
    ba.fit_spectrum(i, use_cstat=False, generic_model="cflux*po", setPars={1:"15,-1", 2:"150,-1",3:-8, 4:"2",5:"1e-3,-1"}, fit_iterations=10000)
    i.plot()

We can, at the very least, use the data to place flux upper limits on the LMXB source. Lets use the comprehensive `batspectrum_analysis` function which automatically calculates the upper limit if the spectrum is not well fit. Since we do not define `nprocs`, it gets set to 1 automatically which means that parallelization is disabled. 

In [ ]:
upperlim_spectra=ba.parallel.batspectrum_analysis(spectra, use_cstat=False, generic_model="cflux*po", setPars={1:"15,-1", 2:"150,-1",3:-8, 4:"2",5:"1e-3,-1"}, fit_iterations=10000, ul_pl_index=1, recalc=True)


1 spectrum  in use
 
Spectral Data File: t_743947552.0-743947553.0_80chan_bkgnsigma_5_upperlim.pha  Spectrum 1
Net count rate (cts/s) for Spectrum:1  4.708e-01 +/- 3.058e-02
 Assigned to Data Group 1 and Plot Group 1
  Noticed Channels:  1-80
  Telescope: SWIFT Instrument: BAT  Channel Type: PI
  Exposure Time: 1 sec
 Using fit statistic: chi
 Using Response (RMF) File            t_743947552.0-743947553.0_80chan.rsp for Source 1


Fit statistic  : Chi-Squared                44076.68     using 80 bins.

Test statistic : Chi-Squared                44076.68     using 80 bins.
 Null hypothesis probability of 0.00e+00 with 78 degrees of freedom
 Current data and model not fit yet.

Model powerlaw<1> Source No.: 1   Active/On
Model Model Component  Parameter  Unit     Value
 par  comp
   1    1   powerlaw   PhoIndex            1.00000      +/-  0.0          
   2    1   powerlaw   norm                1.00000      +/-  0.0          
___________________________________________________________


1 spectrum  in use
 
Spectral Data File: t_743947553.0-743947554.0_80chan_bkgnsigma_5_upperlim.pha  Spectrum 1
Net count rate (cts/s) for Spectrum:1  4.766e-01 +/- 2.993e-02
 Assigned to Data Group 1 and Plot Group 1
  Noticed Channels:  1-80
  Telescope: SWIFT Instrument: BAT  Channel Type: PI
  Exposure Time: 1 sec
 Using fit statistic: chi
 Using Response (RMF) File            t_743947553.0-743947554.0_80chan.rsp for Source 1


Fit statistic  : Chi-Squared                44316.00     using 80 bins.

Test statistic : Chi-Squared                44316.00     using 80 bins.
 Null hypothesis probability of 0.00e+00 with 78 degrees of freedom
 Current data and model not fit yet.

Model powerlaw<1> Source No.: 1   Active/On
Model Model Component  Parameter  Unit     Value
 par  comp
   1    1   powerlaw   PhoIndex            1.00000      +/-  0.0          
   2    1   powerlaw   norm                1.00000      +/-  0.0          
___________________________________________________________


1 spectrum  in use
 
Spectral Data File: t_743947554.0-743947555.0_80chan_bkgnsigma_5_upperlim.pha  Spectrum 1
Net count rate (cts/s) for Spectrum:1  4.811e-01 +/- 3.033e-02
 Assigned to Data Group 1 and Plot Group 1
  Noticed Channels:  1-80
  Telescope: SWIFT Instrument: BAT  Channel Type: PI
  Exposure Time: 1 sec
 Using fit statistic: chi
 Using Response (RMF) File            t_743947554.0-743947555.0_80chan.rsp for Source 1


Fit statistic  : Chi-Squared                43674.70     using 80 bins.

Test statistic : Chi-Squared                43674.70     using 80 bins.
 Null hypothesis probability of 0.00e+00 with 78 degrees of freedom
 Current data and model not fit yet.

Model powerlaw<1> Source No.: 1   Active/On
Model Model Component  Parameter  Unit     Value
 par  comp
   1    1   powerlaw   PhoIndex            1.00000      +/-  0.0          
   2    1   powerlaw   norm                1.00000      +/-  0.0          
___________________________________________________________


1 spectrum  in use
 
Spectral Data File: t_743947555.0-743947556.0_80chan_bkgnsigma_5_upperlim.pha  Spectrum 1
Net count rate (cts/s) for Spectrum:1  4.749e-01 +/- 3.034e-02
 Assigned to Data Group 1 and Plot Group 1
  Noticed Channels:  1-80
  Telescope: SWIFT Instrument: BAT  Channel Type: PI
  Exposure Time: 1 sec
 Using fit statistic: chi
 Using Response (RMF) File            t_743947555.0-743947556.0_80chan.rsp for Source 1


Fit statistic  : Chi-Squared                43911.25     using 80 bins.

Test statistic : Chi-Squared                43911.25     using 80 bins.
 Null hypothesis probability of 0.00e+00 with 78 degrees of freedom
 Current data and model not fit yet.

Model powerlaw<1> Source No.: 1   Active/On
Model Model Component  Parameter  Unit     Value
 par  comp
   1    1   powerlaw   PhoIndex            1.00000      +/-  0.0          
   2    1   powerlaw   norm                1.00000      +/-  0.0          
___________________________________________________________


1 spectrum  in use
 
Spectral Data File: t_743947556.0-743947557.0_80chan_bkgnsigma_5_upperlim.pha  Spectrum 1
Net count rate (cts/s) for Spectrum:1  4.779e-01 +/- 2.958e-02
 Assigned to Data Group 1 and Plot Group 1
  Noticed Channels:  1-80
  Telescope: SWIFT Instrument: BAT  Channel Type: PI
  Exposure Time: 1 sec
 Using fit statistic: chi
 Using Response (RMF) File            t_743947556.0-743947557.0_80chan.rsp for Source 1


Fit statistic  : Chi-Squared                44773.03     using 80 bins.

Test statistic : Chi-Squared                44773.03     using 80 bins.
 Null hypothesis probability of 0.00e+00 with 78 degrees of freedom
 Current data and model not fit yet.

Model powerlaw<1> Source No.: 1   Active/On
Model Model Component  Parameter  Unit     Value
 par  comp
   1    1   powerlaw   PhoIndex            1.00000      +/-  0.0          
   2    1   powerlaw   norm                1.00000      +/-  0.0          
___________________________________________________________


1 spectrum  in use
 
Spectral Data File: t_743947557.0-743947558.0_80chan_bkgnsigma_5_upperlim.pha  Spectrum 1
Net count rate (cts/s) for Spectrum:1  4.747e-01 +/- 2.978e-02
 Assigned to Data Group 1 and Plot Group 1
  Noticed Channels:  1-80
  Telescope: SWIFT Instrument: BAT  Channel Type: PI
  Exposure Time: 1 sec
 Using fit statistic: chi
 Using Response (RMF) File            t_743947557.0-743947558.0_80chan.rsp for Source 1


Fit statistic  : Chi-Squared                44667.82     using 80 bins.

Test statistic : Chi-Squared                44667.82     using 80 bins.
 Null hypothesis probability of 0.00e+00 with 78 degrees of freedom
 Current data and model not fit yet.

Model powerlaw<1> Source No.: 1   Active/On
Model Model Component  Parameter  Unit     Value
 par  comp
   1    1   powerlaw   PhoIndex            1.00000      +/-  0.0          
   2    1   powerlaw   norm                1.00000      +/-  0.0          
___________________________________________________________

Lets plot the upper limits to see what they look like. We can see that the simple powerlaw fit to obtain the flux upper limit is not too bad.

In [ ]:
for i in upperlim_spectra:
    i.plot()

Finally, lets plot the lightcurve and the flux upper limits all together. This is easily done with the `plot_TTE_lightcurve` function. We cna modify the returned axes to make the plot look nicer. 

In [ ]:
fig, ax = ba.plot_TTE_lightcurve(lc, upperlim_spectra, plot_relative=True, T0=event.trigtime.met)
ax[0].axhline(0, ls="--", color="k")
ax[1].set_ylim([1e-7, 9e-7])
fig.savefig(f"{event.result_dir.joinpath("exo_lc.pdf")}", bbox_inches="tight")